# Икс-точка

In [1]:
from IPython.lib.display import YouTubeVideo
YouTubeVideo('2tEHZEMKjHc')

In [2]:
from time import sleep
from collections import deque
from copy import deepcopy
from plotly import graph_objects as go
import ipywidgets as widgets

In [3]:
class Game:
    def __init__(self, player_1, player_2):
        self.player_1 = {'name': player_1[0], 'type': player_1[1], 'symbol': player_1[2], 'role': 'MAX'}
        self.player_2 = {'name': player_2[0], 'type': player_2[1], 'symbol': player_2[2], 'role': 'MIN'}
        self.next_player = self.player_1
        
        self.symbols_fig = {'x': 'x', 'o': 'circle', '·': 'circle-open'}
        self.scores = {'x': 1, 'o': -1, 'draw': 0}
        
        self.score = widgets.HTML(description='Статус:', value='')
        self.state = [['·', '·', '·'], ['·', '·', '·'], ['·', '·', '·']]
        self.fig = self.create_fig()
        self.button = widgets.Button(description='Ресетирај')
        self.button.on_click(lambda x: self.reset())
        display(widgets.VBox([widgets.HBox([self.button, self.score]), self.fig]))
        self.reset()
    
    def reset(self):
        self.next_player = self.player_1
        self.update_score('На ред е', self.next_player)
        self.state = [['·', '·', '·'], ['·', '·', '·'], ['·', '·', '·']]
        self.fig.data[0].marker.symbol = [self.symbols_fig[value] for row in self.state for value in row]
        self.evaluated = {}
        self.take_turns()
    
    def player_took_turn(self, x, y):
        self.state[y][x] = self.next_player['symbol']
        self.fig.data[0].marker.symbol = [self.symbols_fig[value] for row in self.state for value in row]
        winner = self.check_victory(self.state)
        if winner != 'keep_playing':
            if winner == 'draw':
                self.score.value = 'Нерешено.'
            else:
                self.update_score('Победник е', self.next_player)
            return
        self.next_player = self.player_1 if self.next_player == self.player_2 else self.player_2
        self.update_score('На ред е', self.next_player)
        self.take_turns()
    
    def take_turns(self):
        if self.next_player['type'] == 'human':
            return  # wait for player
        else:
            self.score.value += ' -- пресметува'
            state = tuple([tuple(row) for row in self.state])
            result, move = self.minimax(state, self.next_player['role'])
            sleep(1)
            self.player_took_turn(*move)
    
    def update_score(self, message, player):
        player_data = ' - '.join(list(player.values())[:-1])
        self.score.value = f'{message} <b> {player_data} </b>.'
    
    def update_fig(self, trace, points, selector):
        x, y = points.xs[0], points.ys[0]
        if self.state[y][x] == '·' and self.next_player['type'] == 'human':
            self.player_took_turn(x, y)

    def create_fig(self):
        N = 3
        fig = go.FigureWidget()
        x = [x for y in range(N) for x in range(N)]
        y = [y for y in range(N) for x in range(N)]
        symbols = [self.symbols_fig[value] for row in self.state for value in row]
        fig.add_scatter(x=x, y=y, mode='markers', marker_size=48, 
                        marker_symbol=symbols, marker_color='LightSkyBlue',
                        marker_line_width=6, marker_line_color='MediumPurple')
        fig.data[0].on_click(self.update_fig)
        fig.update_xaxes(range=[-0.5, N - 0.5], dtick=1, title='x', side='top')
        fig.update_yaxes(range=[-0.5, N - 0.5], dtick=1, title='y', autorange='reversed')
        fig.update_layout(width=600, height=600, showlegend=False)
        return fig
    
    def minimax(self, node, player, alpha=-2, beta=2, depth=0):
        if node not in self.evaluated:
            self.evaluated[node] = self.check_victory(node)
        if self.evaluated[node] != 'keep_playing':
            return self.scores[self.evaluated[node]], None
        best_value = 2 if player == 'MIN' else -2
        best_move = None
        for child, move in self.expand_state(node, player):
            other_player = 'MIN' if player == 'MAX' else 'MAX'
            result, _ = self.minimax(child, other_player, alpha, beta, depth + 1)
            if player == 'MIN':
                if result <= alpha:
                    return result, best_move
                if result < beta:
                    beta = result
                if result < best_value:
                    best_value = result
                    best_move = move
            elif player == 'MAX':
                if result >= beta:
                    return result, best_move
                if result > alpha:
                    alpha = result
                if result > best_value:
                    best_value = result
                    best_move = move
        return best_value, best_move
    
    def expand_state(self, state, player):
        symbol = 'x' if player == 'MAX' else 'o'
        for y, row in enumerate(state):
            for x, value in enumerate(row):
                if value == '·':
                    new_state = list([list(row) for row in state])
                    new_state[y][x] = symbol
                    yield tuple([tuple(row) for row in new_state]), (x, y)
    
    def check_victory(self, state):
        # проверуваме редици
        for y in range(len(state)):
            if state[y][0] == state[y][1] == state[y][2] and state[y][0] != '·':
                return state[y][0]
        # проверуваме колони
        for x in range(len(state[0])):
            if state[0][x] == state[1][x] == state[2][x] and state[0][x] != '·':
                return state[0][x]
        # проверуваме главна дијагонила
        if state[0][0] == state[1][1] == state[2][2] and state[0][0] != '·':
            return state[0][0]
        # проверуваме споредна дијагонила
        if state[0][2] == state[1][1] == state[2][0] and state[0][2] != '·':
            return state[0][2]
        # проверуваме дали сѐ е пополнето
        if all([value != '·' for row in state for value in row]):
            return 'draw'
        return 'keep_playing'
    

game = Game(('1', 'AI', 'x'), ('2', 'human', 'o'))